In [1]:
import warnings
warnings.filterwarnings('ignore')
import flowermd
import hoomd
from flowermd.library import PPS, OPLS_AA, BeadSpring
from flowermd import Pack
from flowermd.modules.welding import SlabSimulation, Interface, WeldSimulation
from cmeutils.visualize import FresnelGSD
import gsd
import matplotlib.pyplot as plt
import numpy as np
import pickle
import unyt as u
molecule = PPS(num_mols=30, lengths=12)
cpu = hoomd.device.CPU() # So I don't get a CUDA error on local machine

# Create a slab
system = Pack(
    molecules=molecule,
    density=1.1 * u.g/u.cm**3,
)
system.apply_forcefield(r_cut=2.5, force_field=OPLS_AA(), auto_scale=True,remove_charges=True, remove_hydrogens=True)
sim = SlabSimulation.from_system(
    system=system,
    interface_axis=(1,0,0),
    gsd_file_name="slab_x_interface.gsd", device = cpu
)
sim.forces
for wall in sim.forces[-1].walls:
    print(wall)
print()
print("Simulation volume:", sim.box_lengths_reduced)
target_box = flowermd.utils.get_target_box_mass_density(density=1.2*u.g/u.cm**3, mass=sim.mass.to("g"))

sim.run_update_volume(final_box_lengths=target_box, n_steps=5e4, kT=5.0, period=100, tau_kt=0.001)
sim.run_NVT(kT=5.0, n_steps=4e4, tau_kt=0.001)
sim.remove_walls(wall_axis=(1,0,0))
sim.pickle_forcefield()
sim.flush_writers()
sim_viewer = FresnelGSD(gsd_file="slab_x_interface.gsd", view_axis=(0, 1, 0), frame=-1)
sim_viewer.view()

# Create interface from the slab
interface = Interface(gsd_files=["slab_x_interface.gsd"], interface_axis=(1, 0, 0), gap=0.05)
print(type(interface.hoomd_snapshot))
print("Slab number of particles:", system.n_particles)
print("Interface number of particles:", interface.hoomd_snapshot.particles.N)
print()
print("Slab box lengths:", sim.box_lengths_reduced)
print("Interface box lengths:", interface.hoomd_snapshot.configuration.box[:3])

/home/jbieri/miniforge3/envs/flowerMD/lib/python3.11/site-packages/mdtraj/formats/__init__.py:13: DeprecationWarning: 'xdrlib' is deprecated and slated for removal in Python 3.13
  from mdtraj.formats.trr import TRRTrajectoryFile


ParameterizationError: No parameters found for connection <Angle Angle, id: 140232775092256> , group: angle_type, identifiers: [['opls_145', 'opls_202', 'opls_145'], ['CA', 'S', 'CA']] in the Forcefield.

In [ ]:
# Create interface from the slab
interface = Interface(gsd_files=["slab_x_interface.gsd"], interface_axis=(1, 0, 0), gap=0.05)
print(type(interface.hoomd_snapshot))
print("Slab number of particles:", system.n_particles)
print("Interface number of particles:", interface.hoomd_snapshot.particles.N)
print()
print("Slab box lengths:", sim.box_lengths_reduced)
print("Interface box lengths:", interface.hoomd_snapshot.configuration.box[:3])

# Running a welding simulation
# Open and load the forcefield picke file
with open("forcefield.pickle", "rb") as f:
    hoomd_forces = pickle.load(f)

# Let's see what is stored in this pickle file
for force in hoomd_forces:
    print(force)

weld_sim = WeldSimulation(
    initial_state=interface.hoomd_snapshot,
    forcefield=hoomd_forces,
    interface_axis=(1, 0, 0),
    gsd_file_name="weld.gsd",
    log_file_name="weld_log.txt",
    log_write_freq=500,
    dt=0.0003, device = cpu
)
weld_sim.forces
for wall in weld_sim.forces[-1].walls:
    print(wall)
print()
print("Simulation volume:", weld_sim.box_lengths_reduced)
weld_sim.run_NVT(kT=10.0, n_steps=7e4, tau_kt=0.001)
cooling_ramp = weld_sim.temperature_ramp(n_steps=2e4, kT_start=10.0, kT_final=2.0)
weld_sim.run_NVT(kT=cooling_ramp, n_steps=2e4, tau_kt=0.001)
weld_sim.save_restart_gsd("weld_restart.gsd")
weld_sim.flush_writers()


In [ ]:
sim_viewer = FresnelGSD(gsd_file="weld.gsd", view_axis=(0, 1, 0), frame=0, height=12)
weld_colors = np.zeros_like(sim_viewer.positions)
weld_colors[:weld_colors.shape[0]//2 + 1] = np.array([0.5, 0.25, 0.5])
weld_colors[weld_colors.shape[0]//2 + 1:] = np.array([0.5, 0.1, 0.1])
sim_viewer.colors = weld_colors
sim_viewer.view(width=500, height=500)

In [ ]:
sim_viewer.frame = -1
sim_viewer.height = 12
sim_viewer.view_axis = (0, 1, 0)
sim_viewer.view(width=500, height=500)

In [ ]:
# Running a tensile test simulation
from flowermd.library.simulations.tensile import Tensile
# Open and load the forcefield picke file
with open("forcefield.pickle", "rb") as f:
    hoomd_forces = pickle.load(f)
    
tensile_sim = Tensile(
    initial_state="weld_restart.gsd",
    forcefield=hoomd_forces,
    tensile_axis=(1,0,0),
    gsd_file_name="tensile.gsd",
    gsd_write_freq=1000,
    log_file_name="tensile_log.txt",
    log_write_freq=500,
    fix_ratio=0.30,
    device = cpu
)
tensile_sim.run_tensile(n_steps=1e5, strain=0.30, period=500, kT=2.0, tau_kt=0.001)
tensile_sim.flush_writers()